In [95]:
#Import Main Dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from unicodedata import normalize

In [20]:
#Read in the emissions data file 
train_file = "Resources/grocery_train.csv/grocery_train.csv"
train_df = pd.read_csv(train_file)
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [21]:
len(train_df)

3000888

In [22]:
# check data types for SQL compatibility 
train_df.dtypes

id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
dtype: object

In [23]:
# convert 'date' column to datetime format for merging with other CSV's 
train_df['date'] = pd.to_datetime(train_df['date'])
train_df.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

In [37]:
# read in the holiday data file 
holiday_file = "resources/holidays_events.csv"
holiday_df = pd.read_csv(holiday_file)
holiday_df.tail()

,date,type,locale,locale_name,description,transferred
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [38]:
len(holiday_df)

350

In [39]:
holiday_df.dtypes

date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object

In [40]:
holiday_df['date'] = pd.to_datetime(holiday_df['date'])
holiday_df.dtypes

date           datetime64[ns]
type                   object
locale                 object
locale_name            object
description            object
transferred              bool
dtype: object

In [41]:
#drop rows that contain specific 'value' in 'column_name'
cleaned_hol = holiday_df[holiday_df['transferred'] == False]

In [42]:
# cleaned_holiday= holiday_df.loc[holiday_df["transferred"] != "FALSE"]
len(cleaned_hol)

338

In [43]:
cleaned_hol.drop_duplicates(subset='date', inplace=True)
len(cleaned_hol)

C:\Users\victo\AppData\Local\Temp\ipykernel_38904\415807744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_hol.drop_duplicates(subset='date', inplace=True)


300

In [44]:
# drop transferred, cluster and type_x column 
holiday_df = cleaned_hol.drop(['type', "locale_name", "transferred" ], axis=1)
holiday_df.head()

,date,locale,description
0,2012-03-02,Local,Fundacion de Manta
1,2012-04-01,Regional,Provincializacion de Cotopaxi
2,2012-04-12,Local,Fundacion de Cuenca
3,2012-04-14,Local,Cantonizacion de Libertad
4,2012-04-21,Local,Cantonizacion de Riobamba


In [45]:
merged_holiday = pd.merge(train_df, holiday_df, on ='date', how="left")
merged_holiday.head()

,id,date,store_nbr,family,sales,onpromotion,locale,description
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,National,Primer dia del ano
1,1,2013-01-01,1,BABY CARE,0.0,0,National,Primer dia del ano
2,2,2013-01-01,1,BEAUTY,0.0,0,National,Primer dia del ano
3,3,2013-01-01,1,BEVERAGES,0.0,0,National,Primer dia del ano
4,4,2013-01-01,1,BOOKS,0.0,0,National,Primer dia del ano


In [47]:
#Read in the emissions data file 
oil_file = "Resources/oil.csv"
oil_df = pd.read_csv(oil_file)
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [53]:
cleaned_oil = oil_df.dropna()
cleaned_oil.head()

,date,dcoilwtico
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21


In [55]:
# convert 'date' column to datetime format for merging with other CSV's 
cleaned_oil['date'] = pd.to_datetime(cleaned_oil['date'])
cleaned_oil.dtypes

C:\Users\victo\AppData\Local\Temp\ipykernel_38904\3050356060.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_oil['date'] = pd.to_datetime(cleaned_oil['date'])


date          datetime64[ns]
dcoilwtico           float64
dtype: object

In [111]:
final_train = pd.merge(merged_holiday, cleaned_oil, on ='date', how="left")
final_train.head()

,id,date,store_nbr,family,sales,onpromotion,locale,description,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,National,Primer dia del ano,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,National,Primer dia del ano,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,National,Primer dia del ano,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,National,Primer dia del ano,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,National,Primer dia del ano,NaN


Victor Edit

In [112]:
# final_train.loc[final_train["dcoilwtico"] <= 100] = 1
# final_train.loc[final_train["dcoilwtico"] > 100] = 0

In [110]:
# final_train = final_train["dcoilwtico"].dropna()
# final_train

# final_train.drop(final_train[final_train['dcoilwtico'] >= 24000].index, inplace = True)

0          1
1          1
2          1
3          1
4          1
          ..
3000883    1
3000884    1
3000885    1
3000886    1
3000887    1
Name: dcoilwtico, Length: 433026, dtype: int32

In [107]:
final_train["dcoilwtico"] = np.where(final_train["dcoilwtico"] > 100.00, 0, 1)
display(final_train)

,id,date,store_nbr,family,sales,onpromotion,locale,description,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,National,Primer dia del ano,1
1,1,2013-01-01,1,BABY CARE,0.000,0,National,Primer dia del ano,1
2,2,2013-01-01,1,BEAUTY,0.000,0,National,Primer dia del ano,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,National,Primer dia del ano,1
4,4,2013-01-01,1,BOOKS,0.000,0,National,Primer dia del ano,1
...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Local,Fundacion de Riobamba,1
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Local,Fundacion de Riobamba,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Local,Fundacion de Riobamba,1
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Local,Fundacion de Riobamba,1


In [108]:
print(final_train['dcoilwtico'].value_counts())

1    433026
Name: dcoilwtico, dtype: int64
